Screen contaminants of interest. GAMA top 10 + benzene, toluene, ethylbenzene, xylene

Group by Well ID, group by date, keep only rows with all contaminants

In [1]:
from pathlib import Path
import os
import modin.pandas as pd

In [2]:
bp = Path(os.getcwd())

In [3]:
def mkdir_except(folder_name):

    # try to make directory (folder_name), if it exists, ignore
    try:
        # create a directory and print the path
        os.mkdir(folder_name)
        print("Directory {} created".format(folder_name))

    except:
        print("Directory {} already exists".format(folder_name))

In [4]:
def sid_col(df):
    df['SID'] = df['WID'].astype(
        str) + ' ' + df['LOGDATE'].astype(str) + ' ' + df['PARLABEL'].astype(str)

In [5]:
def clean_filter_gama_resutls(df):

    column_dict = {'GM_WELL_ID': 'WID', 'GM_CHEMICAL_VVL': 'PARLABEL','GM_RESULT_MODIFIER': 'PARVQ','GM_RESULT': 'PARVAL','GM_REPORTING_LIMIT':'REPDL',
            'GM_SAMP_COLLECTION_DATE': 'LOGDATE', 'GM_RESULT_UNITS': 'UNITS'}

    df = df.rename(columns=column_dict)

    columns = ['WID', 'PARLABEL', 'PARVQ', 'PARVAL', 'REPDL', 'LOGDATE', 'UNITS', 'SID']

    '''df = df[
        (df['LOGDATE'] >= date) &
        (df['PARVQ'] != '<') &
        (df['PARVAL'] != 0) &
        (df['PARVAL'] > df['REPDL'])
    ]'''

    sid_col(df)
    df = df[columns]
    return df

In [14]:
def create_gama_table(p): # loads gama results data to dataframe

    print('Loading: {} '.format(p))

    try:
        print('try 1')
        df = pd.read_csv(p, sep='\t', lineterminator='\n', encoding='unicode_escape', low_memory=False, parse_dates=['GM_SAMP_COLLECTION_DATE'])
        print('Renaming columns and creating SIDs: \n')
        #df = clean_filter_gama_resutls(df)
        
    except:
        print('except 1')
        try:
            print('try 2')
            df = pd.read_csv(p, sep='\t', lineterminator='\n', low_memory=False, parse_dates=['GM_SAMP_COLLECTION_DATE'])
            print('Renaming columns and creating SIDs: \n')
            #df = clean_filter_gama_resutls(df)

        except:
            print('except 2')
            print('Corrupt: {} '.format(p))
            df = None

    return df

In [7]:
def concat_gama_data(files): # function to concat gama result datasets

    df_list = []

    for i in files:
        j = create_gama_table(i)
        if j is not None:
            df_list.append(j)

    print('\nCombining GAMA sample results: \n')

    concatDF = pd.concat(df_list, axis=0)
    for df in df_list:
        del df
    return concatDF

In [15]:
gama_res_path = bp / 'gama_results'
mkdir_except(gama_res_path)

# collect gama results files and concat them
print('Loading GAMA sample results: \n')
gama_files = gama_res_path.glob('**/*.zip')
gama_results = concat_gama_data(gama_files)

Directory c:\Users\chief\Desktop\sample_filtering\gama_results already exists
Loading GAMA sample results: 

Loading: c:\Users\chief\Desktop\sample_filtering\gama_results\gama_ddw_alameda_v2.zip 
try 1
Renaming columns and creating SIDs: 

Loading: c:\Users\chief\Desktop\sample_filtering\gama_results\gama_ddw_contracosta_v2.zip 
try 1
Renaming columns and creating SIDs: 

Loading: c:\Users\chief\Desktop\sample_filtering\gama_results\gama_ddw_delnorte_v2.zip 
try 1
Renaming columns and creating SIDs: 

Loading: c:\Users\chief\Desktop\sample_filtering\gama_results\gama_ddw_humboldt_v2.zip 
try 1
Renaming columns and creating SIDs: 

Loading: c:\Users\chief\Desktop\sample_filtering\gama_results\gama_ddw_losangeles_v2.zip 
try 1
Renaming columns and creating SIDs: 

Loading: c:\Users\chief\Desktop\sample_filtering\gama_results\gama_ddw_marin_v2.zip 
try 1
Renaming columns and creating SIDs: 

Loading: c:\Users\chief\Desktop\sample_filtering\gama_results\gama_ddw_mendocino_v2.zip 
try 1
Ren

In [32]:
test = gama_results.groupby(['GM_WELL_ID', 'GM_SAMP_COLLECTION_DATE'])

In [ ]:
test.head(5)

In [9]:
def clean_filter_edf_resutls(df):

    columns = ['WID', 'PARLABEL', 'PARVQ', 'PARVAL', 'REPDL', 'LOGDATE', 'UNITS', 'SID']

    '''df = df[
        (df['LOGDATE'] >= date) &
        (df['PARVQ'] != '<') &
        (df['PARVAL'] != 0) &
        (df['PARVAL'] > df['REPDL'])
    ]'''
    
    df['WID'] = df['GLOBAL_ID'] + '-' + df['FIELD_PT_NAME']
    sid_col(df)
    df = df[columns]
    return df

In [16]:
def create_edf_table(p):

    print('Loading: {} '.format(p))

    try:
        print('try 1')
        df = pd.read_csv(p, sep='\t', lineterminator='\n', encoding='unicode_escape', parse_dates=[
            'LOGDATE'])
        print('Renaming columns and creating SIDs: \n')
        df = clean_filter_edf_resutls(df)

    except:
        print('except 1')
        try:
            print('try 2')
            df = pd.read_csv(p, sep='\t', lineterminator='\n', parse_dates=[
                'LOGDATE'])
            print('Renaming columns and creating SIDs: \n')
            df = clean_filter_edf_resutls(df)

        except:
            print('except 2')
            print('Corrupt: {} '.format(p))
            df = None

    return df

In [11]:

def concat_geo_data(files): # function to concat gama result datasets

    df_list = []

    for i in files:
        j = create_edf_table(i)

        if j is not None:
            df_list.append(j)

    print('Combining Geotracker EDF sample results: \n')

    concat_df = pd.concat(df_list, axis=0)

    for df in df_list:
        del df

    return concat_df

In [12]:
# Create the path for geotracker downloads (edf AND xy)
geo_edf_path = bp / 'geotracker_edf_results'

# Call function to create directories for geotracker downloads.
mkdir_except(geo_edf_path)

# Load and combine Geotracker EDF results
print('Loading Geotracker EDF results \n')

edf_files = geo_edf_path.glob('**/*.zip')
edf_results = concat_geo_data(edf_files)

Directory c:\Users\chief\Desktop\sample_filtering\geotracker_edf_results already exists
Loading Geotracker EDF results 

Loading: c:\Users\chief\Desktop\sample_filtering\geotracker_edf_results\AlamedaEDF.zip 


C:\Users\chief\AppData\Local\Temp\ipykernel_2968\1253246279.py:6: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  j = create_edf_table(i)


Renaming columns and creating SIDs: 

Corrupt: c:\Users\chief\Desktop\sample_filtering\geotracker_edf_results\AlamedaEDF.zip 
Loading: c:\Users\chief\Desktop\sample_filtering\geotracker_edf_results\ContraCostaEDF.zip 
Renaming columns and creating SIDs: 

Renaming columns and creating SIDs: 

Corrupt: c:\Users\chief\Desktop\sample_filtering\geotracker_edf_results\ContraCostaEDF.zip 
Loading: c:\Users\chief\Desktop\sample_filtering\geotracker_edf_results\DelNorteEDF.zip 
Renaming columns and creating SIDs: 

Renaming columns and creating SIDs: 

Corrupt: c:\Users\chief\Desktop\sample_filtering\geotracker_edf_results\DelNorteEDF.zip 
Loading: c:\Users\chief\Desktop\sample_filtering\geotracker_edf_results\HumboldtEDF.zip 
Renaming columns and creating SIDs: 

Renaming columns and creating SIDs: 

Corrupt: c:\Users\chief\Desktop\sample_filtering\geotracker_edf_results\HumboldtEDF.zip 
Loading: c:\Users\chief\Desktop\sample_filtering\geotracker_edf_results\LosAngelesEDF.zip 
Renaming column

ValueError: No objects to concatenate